# Final Model Preprocessing Pipeline

Complete pipeline: PDF Schedule → Predictions CSV

**Input:** `Data/data_v1/25-26/GW Spielplan 2025-2026-1.pdf`
**Output:** `Data/data_v1/25-26/2025-26_predictions.csv`

In [1]:
import pandas as pd
import numpy as np
import json
import pdfplumber
import holidays
import joblib
from catboost import CatBoostRegressor
from datetime import datetime
import re

## 1. PDF Parsing

In [2]:
# Configuration
PDF_PATH = 'Data/data_v1/25-26/GW Spielplan 2025-2026-1.pdf'
OUTPUT_PATH = 'Data/data_v1/25-26/2025-26_predictions.csv'

# Arena capacity and ticket price for revenue calculation
ARENA_CAPACITY = 4000
TICKET_PRICE = 25.0

In [3]:
# Extract schedule from PDF
def extract_schedule_from_pdf(pdf_path):
    games = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            
            for table in tables:
                for row in table:
                    if row and len(row) >= 6:
                        # Skip header rows
                        if row[0] and (row[0].strip() == '#' or 'Spieltag' in str(row[0])):
                            continue
                        
                        try:
                            # Parse row: [#, Date, Weekday, Face-Off, Home, Away, Distance]
                            spieltag = row[0].strip() if row[0] else ''
                            date_str = row[1].strip() if row[1] else ''
                            weekday = row[2].strip() if row[2] else ''
                            time_str = row[3].strip() if row[3] else ''
                            home = row[4].strip() if row[4] else ''
                            away = row[5].strip() if row[5] else ''
                            distance = row[6].strip() if len(row) > 6 and row[6] else ''
                            
                            # Skip empty or invalid rows
                            if not spieltag or not date_str or not home:
                                continue
                            
                            games.append({
                                'spieltag': spieltag,
                                'date': date_str,
                                'weekday': weekday,
                                'time': time_str,
                                'home_team': home,
                                'away_team': away,
                                'distance': distance
                            })
                        except (IndexError, AttributeError):
                            continue
    
    return pd.DataFrame(games)

# Extract from PDF
df_raw = extract_schedule_from_pdf(PDF_PATH)
print(f"Extracted {len(df_raw)} rows from PDF")
df_raw.head(10)

Extracted 52 rows from PDF


,spieltag,date,weekday,time,home_team,away_team,distance
0,1,12.09.2025,Fri,19:30,Löwen Frankfurt (GER),Grizzlys Wolfsburg (GER),369 km
1,2,14.09.2025,Sun,16:30,Grizzlys Wolfsburg (GER),Nürnberg Ice Tigers (GER),463 km
2,3,19.09.2025,Fri,19:30,ERC Ingolstadt (GER),Grizzlys Wolfsburg (GER),526 km
3,4,21.09.2025,Sun,16:30,Grizzlys Wolfsburg (GER),Dresden Eislöwen (GER),367 km
4,5,26.09.2025,Fri,19:30,Grizzlys Wolfsburg (GER),Schwenninger Wild Wings (GER),638 km
5,6,28.09.2025,Sun,16:30,Kölner Haie (GER),Grizzlys Wolfsburg (GER),376 km
6,7,03.10.2025,Fri,16:30,EHC Red Bull München (GER),Grizzlys Wolfsburg (GER),600 km
7,8,05.10.2025,Sun,14:00,Grizzlys Wolfsburg (GER),Pinguins Bremerhaven (GER),252 km (D)
8,9,10.10.2025,Fri,19:30,Augsburger Panther (GER),Grizzlys Wolfsburg (GER),588 km
9,10,12.10.2025,Sun,16:30,Grizzlys Wolfsburg (GER),Straubing Tigers (GER),560 km


## 2. Data Cleaning

In [4]:
# Team name mappings
TEAM_NAME_MAP = {
    'Grizzlys': 'Grizzlys Wolfsburg',
    'Wolfsburg': 'Grizzlys Wolfsburg',
    'Bremerhaven': 'Pinguins Bremerhaven',
    'München': 'EHC Red Bull München',
    'Berlin': 'Eisbären Berlin',
    'Düsseldorf': 'Düsseldorfer EG',
    'Köln': 'Kölner Haie',
    'Frankfurt': 'Löwen Frankfurt',
    'Nürnberg': 'Nürnberg Ice Tigers',
    'Mannheim': 'Adler Mannheim',
    'Augsburg': 'Augsburger Panther',
    'Ingolstadt': 'ERC Ingolstadt',
    'Iserlohn': 'Iserlohn Roosters',
    'Schwenningen': 'Schwenninger Wild Wings',
    'Straubing': 'Straubing Tigers',
    'Bietigheim': 'SC Bietigheim Steelers'
}

# Distance mapping (in km)
DISTANCE_MAP = {
    'Pinguins Bremerhaven': 252,
    'Eisbären Berlin': 228,
    'Iserlohn Roosters': 304,
    'Kölner Haie': 376,
    'Löwen Frankfurt': 369,
    'Adler Mannheim': 442,
    'Nürnberg Ice Tigers': 463,
    'ERC Ingolstadt': 526,
    'Straubing Tigers': 560,
    'Augsburger Panther': 588,
    'EHC Red Bull München': 600,
    'Schwenninger Wild Wings': 638,
    'Düsseldorfer EG': 350,
    'SC Bietigheim Steelers': 480
}

def clean_team_name(name):
    """Clean and normalize team name"""
    if not name:
        return name
    
    # Remove country codes
    name = re.sub(r'\s*\(GER\)\s*', '', name).strip()
    
    # Apply mapping
    for key, value in TEAM_NAME_MAP.items():
        if key.lower() in name.lower():
            return value
    
    return name

def parse_distance(dist_str):
    """Parse distance string to numeric"""
    if not dist_str:
        return None
    # Extract numeric value
    match = re.search(r'(\d+)', str(dist_str))
    if match:
        return int(match.group(1))
    return None

In [5]:
# Clean the data
df = df_raw.copy()

# Clean team names
df['home_team'] = df['home_team'].apply(clean_team_name)
df['away_team'] = df['away_team'].apply(clean_team_name)

# Parse spieltag as integer
df['spieltag'] = pd.to_numeric(df['spieltag'], errors='coerce')

# Parse distance
df['distance'] = df['distance'].apply(parse_distance)

# Backfill missing distances from mapping
df['distance'] = df.apply(
    lambda row: row['distance'] if pd.notna(row['distance']) else DISTANCE_MAP.get(row['away_team'], 400),
    axis=1
)

# Filter to home games only
df = df[df['home_team'] == 'Grizzlys Wolfsburg'].copy()

# Add season
df['season'] = '25-26'

# Drop rows with missing critical data
df = df.dropna(subset=['spieltag', 'date', 'time'])

print(f"Cleaned data: {len(df)} home games")
df.head(10)

Cleaned data: 26 home games


,spieltag,date,weekday,time,home_team,away_team,distance,season
1,2,14.09.2025,Sun,16:30,Grizzlys Wolfsburg,Nürnberg Ice Tigers,463,25-26
3,4,21.09.2025,Sun,16:30,Grizzlys Wolfsburg,Dresden Eislöwen,367,25-26
4,5,26.09.2025,Fri,19:30,Grizzlys Wolfsburg,Schwenninger Wild Wings,638,25-26
7,8,05.10.2025,Sun,14:00,Grizzlys Wolfsburg,Pinguins Bremerhaven,252,25-26
9,10,12.10.2025,Sun,16:30,Grizzlys Wolfsburg,Straubing Tigers,560,25-26
11,12,19.10.2025,Sun,14:00,Grizzlys Wolfsburg,Adler Mannheim,442,25-26
12,13,24.10.2025,Fri,19:30,Grizzlys Wolfsburg,Iserlohn Roosters,304,25-26
14,15,28.10.2025,Tue,19:30,Grizzlys Wolfsburg,EHC Red Bull München,600,25-26
15,16,30.10.2025,Thu,19:30,Grizzlys Wolfsburg,Augsburger Panther,588,25-26
17,18,14.11.2025,Fri,19:30,Grizzlys Wolfsburg,Kölner Haie,376,25-26


## 3. Feature Engineering

In [6]:
# Load saved encodings from v7 model
with open('Models/baseline_model_v7/opponent_encoding_v7.json', 'r') as f:
    opponent_encoding = json.load(f)

with open('Models/baseline_model_v7/sunday_multipliers_v7.json', 'r') as f:
    sunday_multipliers = json.load(f)

with open('Models/baseline_model_v7/feature_cols_v7.json', 'r') as f:
    feature_cols = json.load(f)

print(f"Loaded {len(opponent_encoding)} opponent encodings")
print(f"Feature columns: {feature_cols}")

Loaded 14 opponent encodings
Feature columns: ['weekday_sin', 'weekday_cos', 'hour', 'month_sin', 'is_dec_holiday', 'holiday_score', 'spieltag', 'game_progress', 'opponent_attendance', 'distance_log', 'is_top_opponent', 'sunday_boost', 'sunday_opp_adj', 'sunday_top']


In [7]:
# Parse datetime
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%d.%m.%Y %H:%M')

# Extract time features
df['month'] = df['datetime'].dt.month
df['weekday_num'] = df['datetime'].dt.weekday  # 0=Monday, 6=Sunday
df['hour'] = df['datetime'].dt.hour + df['datetime'].dt.minute / 60

# Cyclical encoding
df['weekday_sin'] = np.sin(2 * np.pi * df['weekday_num'] / 7)
df['weekday_cos'] = np.cos(2 * np.pi * df['weekday_num'] / 7)
df['month_sin'] = np.sin(2 * np.pi * (df['month'] - 1) / 12)

print("Time features created")

Time features created


In [8]:
# Sunday score
def calc_sunday_score(row):
    if row['weekday_num'] == 6:  # Sunday
        if 14 <= row['hour'] <= 17:
            return 2.0
        else:
            return 1.0
    return 0.0

df['sunday_score'] = df.apply(calc_sunday_score, axis=1)
print(f"Sunday games: {(df['sunday_score'] > 0).sum()}")

Sunday games: 10


In [9]:
# Holiday features
german_holidays = holidays.Germany(state='NI', years=[2025, 2026])

def calc_holiday_features(row):
    dt = row['datetime']
    date_obj = dt.date()
    
    is_dec_holiday = 1 if (dt.month == 12 and dt.day >= 20) else 0
    
    score = 0
    if date_obj in german_holidays:
        score += 2
    if (dt.month == 12 and dt.day >= 20) or (dt.month == 1 and dt.day <= 6):
        score += 2
    if dt.month == 10 and 4 <= dt.day <= 19:
        score += 1
    
    return pd.Series([is_dec_holiday, score])

df[['is_dec_holiday', 'holiday_score']] = df.apply(calc_holiday_features, axis=1)
print(f"December holiday games: {df['is_dec_holiday'].sum()}")

December holiday games: 3


In [10]:
# Game progress
min_spieltag = df['spieltag'].min()
max_spieltag = df['spieltag'].max()
df['game_progress'] = (df['spieltag'] - min_spieltag) / (max_spieltag - min_spieltag)

print(f"Spieltag range: {min_spieltag} - {max_spieltag}")

Spieltag range: 2 - 51


In [11]:
# Opponent features
global_median = 1547

df['opponent_attendance'] = df['away_team'].map(opponent_encoding).fillna(global_median)
df['distance_log'] = np.log(df['distance'])

top_opponents = ['Eisbären Berlin', 'Düsseldorfer EG', 'Kölner Haie', 'Pinguins Bremerhaven']
df['is_top_opponent'] = df['away_team'].isin(top_opponents).astype(int)

print(f"Top opponent games: {df['is_top_opponent'].sum()}")

Top opponent games: 6


In [12]:
# Interaction features
df['sunday_boost'] = df['sunday_score'] * (df['opponent_attendance'] / global_median)
df['sunday_mult'] = df['away_team'].map(sunday_multipliers).fillna(1.0)
df['sunday_opp_adj'] = df['sunday_score'] * df['sunday_mult']
df['sunday_top'] = df['sunday_score'] * df['is_top_opponent']

print("Interaction features created")

Interaction features created


## 4. Model Inference

In [13]:
# Prepare feature matrix
X = df[feature_cols].fillna(0)

print(f"Feature matrix shape: {X.shape}")
print(f"Features: {list(X.columns)}")

Feature matrix shape: (26, 14)
Features: ['weekday_sin', 'weekday_cos', 'hour', 'month_sin', 'is_dec_holiday', 'holiday_score', 'spieltag', 'game_progress', 'opponent_attendance', 'distance_log', 'is_top_opponent', 'sunday_boost', 'sunday_opp_adj', 'sunday_top']


In [14]:
# Load models
scaler = joblib.load('Models/baseline_model_v7/scaler_v7.joblib')
ridge = joblib.load('Models/baseline_model_v7/ridge_v7.joblib')

catboost = CatBoostRegressor()
catboost.load_model('Models/baseline_model_v7/catboost_v7.cbm')

print("Models loaded successfully")

Models loaded successfully


In [15]:
# Make predictions
X_scaled = scaler.transform(X)
pred_ridge = ridge.predict(X_scaled)
pred_catboost = catboost.predict(X)

# Ensemble average
predictions = (pred_ridge + pred_catboost) / 2

print(f"Predictions range: {predictions.min():.0f} - {predictions.max():.0f}")
print(f"Mean prediction: {predictions.mean():.0f}")

Predictions range: 746 - 2716
Mean prediction: 1647


## 5. Generate Output

In [16]:
# Create output dataframe
output_df = pd.DataFrame({
    'date': df['date'].values,
    'predicted_attendance': np.round(predictions, 2),
    'predicted_revenue': np.round(predictions * TICKET_PRICE, 2),
    'occupancy_rate': np.round(predictions / ARENA_CAPACITY, 4)
})

print(f"Output shape: {output_df.shape}")
output_df

Output shape: (26, 4)


,date,predicted_attendance,predicted_revenue,occupancy_rate
0,14.09.2025,1691.55,42288.64,0.4229
1,21.09.2025,1576.27,39406.84,0.3941
2,26.09.2025,1110.98,27774.62,0.2777
3,05.10.2025,1926.50,48162.57,0.4816
4,12.10.2025,1231.18,30779.41,0.3078
5,19.10.2025,1527.30,38182.49,0.3818
6,24.10.2025,1240.83,31020.79,0.3102
7,28.10.2025,746.43,18660.84,0.1866
8,30.10.2025,914.85,22871.31,0.2287
9,14.11.2025,1506.84,37671.02,0.3767


In [19]:
# print the original dataframe
print(df.head(10))

    spieltag        date weekday   time           home_team  \
1          2  14.09.2025     Sun  16:30  Grizzlys Wolfsburg   
3          4  21.09.2025     Sun  16:30  Grizzlys Wolfsburg   
4          5  26.09.2025     Fri  19:30  Grizzlys Wolfsburg   
7          8  05.10.2025     Sun  14:00  Grizzlys Wolfsburg   
9         10  12.10.2025     Sun  16:30  Grizzlys Wolfsburg   
11        12  19.10.2025     Sun  14:00  Grizzlys Wolfsburg   
12        13  24.10.2025     Fri  19:30  Grizzlys Wolfsburg   
14        15  28.10.2025     Tue  19:30  Grizzlys Wolfsburg   
15        16  30.10.2025     Thu  19:30  Grizzlys Wolfsburg   
17        18  14.11.2025     Fri  19:30  Grizzlys Wolfsburg   

                  away_team  distance season            datetime  month  ...  \
1       Nürnberg Ice Tigers       463  25-26 2025-09-14 16:30:00      9  ...   
3          Dresden Eislöwen       367  25-26 2025-09-21 16:30:00      9  ...   
4   Schwenninger Wild Wings       638  25-26 2025-09-26 19:30:00  

In [20]:
# Save predictions
output_df.to_csv(OUTPUT_PATH, index=False)
print(f"Saved predictions to: {OUTPUT_PATH}")
print(f"\nSummary:")
print(f"  Total games: {len(output_df)}")
print(f"  Avg attendance: {output_df['predicted_attendance'].mean():.0f}")
print(f"  Avg occupancy: {output_df['occupancy_rate'].mean():.1%}")
print(f"  Total revenue: €{output_df['predicted_revenue'].sum():,.0f}")

Saved predictions to: Data/data_v1/25-26/2025-26_predictions.csv

Summary:
  Total games: 26
  Avg attendance: 1647
  Avg occupancy: 41.2%
  Total revenue: €1,070,429


In [21]:
# save it to a json file
output_df.to_json('Data/data_v1/25-26/2025-26_predictions_v2.json', orient='records', lines=True)

In [22]:
# for the output_df predicted_attendance add a constant of 1000 to each prediction to account for season tickets sold
output_df['predicted_attendance'] += 1000

# update the occupancy rate accordingly
output_df['occupancy_rate'] = np.round(output_df['predicted_attendance'] / ARENA_CAPACITY, 4)

print(output_df.head(10))

         date  predicted_attendance  predicted_revenue  occupancy_rate
0  14.09.2025               2691.55           42288.64          0.6729
1  21.09.2025               2576.27           39406.84          0.6441
2  26.09.2025               2110.98           27774.62          0.5277
3  05.10.2025               2926.50           48162.57          0.7316
4  12.10.2025               2231.18           30779.41          0.5578
5  19.10.2025               2527.30           38182.49          0.6318
6  24.10.2025               2240.83           31020.79          0.5602
7  28.10.2025               1746.43           18660.84          0.4366
8  30.10.2025               1914.85           22871.31          0.4787
9  14.11.2025               2506.84           37671.02          0.6267


In [23]:
# save only the date and occupancy_rate to a new json file
output_df[['date', 'occupancy_rate']].to_json('Data/data_v1/25-26/2025-26_occupancy_rates_with_season_tickets.json', orient='records', lines=True)

In [24]:
# load and update 2025-26_predictions_v2.json with a season_included_occupancy_rate field
with open('Data/data_v1/25-26/2025-26_predictions_v2.json', 'r') as f:
    predictions_data = [json.loads(line) for line in f]
for record in predictions_data:
    date = record['date']
    occupancy_rate = output_df.loc[output_df['date'] == date, 'occupancy_rate'].values[0]
    record['season_included_occupancy_rate'] = occupancy_rate

# save updated predictions_data back to json file
with open('Data/data_v1/25-26/2025-26_predictions_v2.json', 'w') as f:
    for record in predictions_data:
        f.write(json.dumps(record) + '\n')